# Author: Yotam Dery
# Date: 03/03/2025

# Part 1 - Prediction task - user income prediction

## Imports

In [ ]:
# Basics
import numpy as np
import pandas as pd
from ydata_profiling import ProfileReport
# ML related
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, GridSearchCV
from scipy.stats import skew
# Projects scripts
from prediction_task_plot_utils import plot_target_label, plot_features_dist, plot_corr_with_label
from prediction_task_utils import check_skewness_score, apply_log_transformation, \
                                    identify_skewed_features, identify_highly_correlated_features,\
                                    get_outliers_top_n_features
import warnings
warnings.simplefilter("ignore")

# Data Loading

In [ ]:
# Loading the train data
df_train = pd.read_csv('train_home_assignment.csv', index_col=0)
# Dropping columns for this task
df_train = df_train.drop(columns=["treatment", "org_price_usd_following_30_days_after_impact"])
print('train shape is: {}'.format(df_train.shape))

# Loading the test data
df_test = pd.read_csv('test_home_assignment.csv', index_col=0)
print('test shape is: {}'.format(df_test.shape))

In [ ]:
# First inspect of the train set
df_train.head()

# Train-validation split

* We'd like to first perfrom the train-validation split to ensure that the needed operations are made using training set statistics, <br>
and to prevent data leakage. <br>
We'll use a 80-20 Split (80% for training, 20% for validation).

In [ ]:
# Define features and target
X = df_train.drop(columns=["org_price_usd_following_30_days"])
y = df_train["org_price_usd_following_30_days"]

# Create separate targets:
y_log = np.log1p(y)  # Log transformation (only for Linear Regression)
y_original = y  # Keep original target for tree-based models

# Split into training & validation sets
X_train, X_val, y_train_log, y_val_log = train_test_split(X, y_log, test_size=0.2, random_state=42)  # Log target for linear
_, _, y_train_original, y_val_original = train_test_split(X, y_original, test_size=0.2, random_state=42)  # Original target for trees

# EDA

In [ ]:
# Let's print some statistics for the train set
X_train.describe()

In [ ]:
y_train_original.describe()

In [ ]:
# Basic structure of the dataset
train_info = X_train.info()
train_info

In [ ]:
validation_info = X_val.info()
validation_info

In [ ]:
# Check for missing values
missing_values = X_train.isnull().sum().sort_values(ascending=False)
missing_values

* <b>Insights from previous steps:</b>
1. Training set: 160,000 rows
Validation set: 40,000 rows 

2. Dataset Structure
53 features (excluding the target variable). <br>
All features are numerical (float64 or int64). <br>
There are no missing values. <br>
Some features have high variance, indicating possible outliers. <br>
Certain features have long tails, meaning a log transformation may help.

In [ ]:
## Plot the target label histogram
# Define fixed bin edges from 0 to 800 with intervals of 50
start, end, step = 0, 850, 50
bin_edges = np.arange(start, end, step)  # 850 ensures last bin covers up to 800
plot_target_label(bin_edges, y_train_original)

* The target label looks very skewed (strong right tail). <br> If we consider models that assume normally distributed residuals (like linear regression), we might want to perform a log transformation to the target label

* <b> Skewness criteria: </b> <br>
Skewness > 1.0 → highly skewed (log transform might help). <br>
Skewness between 0.5 and 1.0 → moderately skewed (consider transforming). <br>
Skewness < 0.5 → nearly symmetric (transformation unnecessary).

In [ ]:
print(check_skewness_score(y_train_original))
print("Looks like the target label is highly skewed before the trasformation and symmetric after!")

In [ ]:
# Plot the transformed target label
y_train_log_transformed = apply_log_transformation(y_train_original)
start, end, step = 0, 11, 1
bin_edges = np.arange(start, end, step)  
plot_target_label(bin_edges, y_train_log_transformed)

## Univariate Analysis (Features Distribution)

In [ ]:
plot_features_dist(X_train)

In [ ]:
identify_skewed_features(X_train)

In [ ]:
# # For a deeper profiling (but takes long time for execution)
# profile = ProfileReport(X_train, explorative=True)
# profile.to_notebook_iframe()

Insights from Univariate Analysis: <br>
1. Many real-world use cases (like user spending, income, and engagement metrics) follow a right-skewed distribution (like we see here!).<br> <br>
2. Some features (e.g., payment_occurrences_preceding_30_days,payment_occurrences_preceding_3_days) have right-skewed distributions, meaning log transformation might help. 

## Bivariate Analysis (Feature Relationships)
In this step, we'll analyze how different features relate to the target variable (tag) to uncover important patterns.

In [ ]:
plot_corr_with_label(X_train, y_train_original)

In [ ]:
# Check for highly correlated pairs of features
identify_highly_correlated_features(X_train)

Here are all pairs of highly correlated features (corr>0.9).<br> Some highly correlated features might capture different patterns (e.g., spending behavior vs. engagement behavior), <br>so we wont drop them for now

## Outliers Detection
* Here we perform the outliers detection for the top 20 highly correlated features (with respect to the target label)

In [ ]:
get_outliers_top_n_features(X_train, y_train_original)

# Data Preprocessing

In [ ]:
class CustomPreprocessor(BaseEstimator, TransformerMixin):
    """
    A custom transformer to preprocess data for machine learning models.
    
    - For Linear Regression:
        - Applies log transformation (with automatic negative value shifting).
        - Removes outliers using IQR clipping.
        - Standardizes features using StandardScaler.
        - Optionally removes highly correlated features.
    
    - For Tree-Based Models (Random Forest, XGBoost):
        - Skips outlier removal (trees handle outliers naturally).
        - Skips scaling (trees do not require feature scaling).
        - Optionally removes highly correlated features.

    Parameters:
        model_type (str): "linear" for Linear Regression, "tree" for RF/XGBoost.
        remove_high_corr_features (bool): If True, removes highly correlated features.

    Methods:
        fit(X, y): Learns dataset statistics.
        transform(X): Applies transformations using learned statistics.
    """

    def __init__(self, model_type="linear", remove_high_corr_features=False):
        self.model_type = model_type
        self.remove_high_corr_features = remove_high_corr_features
        self.log_features = []
        self.scaler = None
        self.iqr_limits = {}
        self.high_corr_features = {}
        self.shift_values = {}  # Stores shift values for negative log-transformed features

    def fit(self, X, y=None):
        """
        Learns dataset statistics for transformations.

        Args:
            X (pd.DataFrame): Training data.
            y (pd.Series, optional): Target variable.

        Returns:
            self
        """

        # Identify skewed features for log transformation (only for Linear Regression)
        if self.model_type == "linear":
            skewness = X.apply(lambda x: skew(x), axis=0)
            self.log_features = skewness[abs(skewness) > 1].index.tolist()

            # Detect and store shift values for negative log-transformed features
            for feature in self.log_features:
                min_value = X[feature].min()
                if min_value <= 0:
                    self.shift_values[feature] = abs(min_value) + 1  # Shift to make values positive

        # Compute IQR thresholds for outlier handling (only for Linear Regression)
        if self.model_type == "linear":
            for feature in X.columns:
                Q1 = X[feature].quantile(0.25)
                Q3 = X[feature].quantile(0.75)
                IQR = Q3 - Q1
                lower_bound = Q1 - 1.5 * IQR
                upper_bound = Q3 + 1.5 * IQR
                self.iqr_limits[feature] = (lower_bound, upper_bound)

        # Compute feature scaling using StandardScaler (only for Linear Regression)
        if self.model_type == "linear":
            self.scaler = StandardScaler()
            self.scaler.fit(X)

        # Identify highly correlated features for removal (if enabled)
        if self.remove_high_corr_features:
            corr_matrix = X.corr()
            high_corr_pairs = set()
            for col in corr_matrix.columns:
                for idx in corr_matrix.index:
                    if col != idx and abs(corr_matrix.loc[idx, col]) > 0.9:
                        high_corr_pairs.add((col, idx))
            self.high_corr_features = list(set([pair[1] for pair in high_corr_pairs]))

        return self

    def transform(self, X):
        """
        Applies transformations to the dataset using the learned statistics.

        Args:
            X (pd.DataFrame): Dataset to transform.

        Returns:
            pd.DataFrame: Transformed dataset.
        """
        X_transformed = X.copy()

        # Apply log transformation (only for Linear Regression)
        if self.model_type == "linear" and self.log_features:
            for feature in self.log_features:
                # Apply the shift if required (to prevent log of negative numbers)
                if feature in self.shift_values:
                    X_transformed[feature] += self.shift_values[feature]
                X_transformed[feature] = np.log1p(X_transformed[feature])

        # Apply outlier handling using IQR clipping (only for Linear Regression)
        if self.model_type == "linear":
            for feature, (lower_bound, upper_bound) in self.iqr_limits.items():
                X_transformed[feature] = np.clip(X_transformed[feature], lower_bound, upper_bound)

        # Apply feature scaling using StandardScaler (only for Linear Regression)
        if self.model_type == "linear" and self.scaler:
            X_transformed = pd.DataFrame(self.scaler.transform(X_transformed), columns=X_transformed.columns)

        # Drop highly correlated features (if enabled)
        if self.remove_high_corr_features and self.high_corr_features:
            X_transformed = X_transformed.drop(columns=self.high_corr_features, errors="ignore")

        return X_transformed

In [ ]:
# Initialize the transformer for Linear Regression (without correlated feature removal)
preprocessor_linear = CustomPreprocessor(model_type="linear", remove_high_corr_features=False)

# Fit and transform for Linear Regression
preprocessor_linear.fit(X_train)
X_train_transformed_linear = preprocessor_linear.transform(X_train)
X_val_transformed_linear = preprocessor_linear.transform(X_val)
# Output transformed training data sample (Linear Model)
X_train_transformed_linear.head()

In [ ]:
# Initialize the transformer for Tree-Based Models (without correlated feature removal)
preprocessor_tree = CustomPreprocessor(model_type="tree", remove_high_corr_features=False)

# Fit and transform for Tree-Based Models
preprocessor_tree.fit(X_train)
X_train_transformed_tree = preprocessor_tree.transform(X_train)
X_val_transformed_tree = preprocessor_tree.transform(X_val)

# Output transformed training data sample (Tree-Based Model)
X_train_transformed_tree.head()

# Modeling

## Create a baseline model

In [ ]:
# Quick check for potential RMSE - without a model
np.sqrt(mean_squared_error(y_val_original, X_val['org_price_usd_preceding_30_days']))

In [ ]:
# Initialize the Linear Regression model
linear_model = LinearRegression()

# Train the model using the preprocessed training data (X_train) and log-transformed target (y_train_log)
linear_model.fit(X_train_transformed_linear, y_train_log)

# Predict on the validation set (output is still in log scale)
y_pred_log = linear_model.predict(X_val_transformed_linear)

# Convert predictions back to original scale using expm1()
y_pred_original = np.expm1(y_pred_log)  # Reverse log1p transformation

# Compute RMSE (Root Mean Squared Error) in the original scale
rmse_linear = np.sqrt(mean_squared_error(y_val_original, y_pred_original))

# Output RMSE
rmse_linear

## Create and tune the tree based models

In [ ]:
# Define the parameter grid
rf_param_grid = {
    "n_estimators": [100, 200],  # Number of trees
    "max_depth": [10, 20],  # Depth of trees
}

# Initialize Random Forest
rf_model = RandomForestRegressor(random_state=42, criterion="squared_error")

# Perform Grid Search with Cross-Validation
rf_grid_search = GridSearchCV(
    rf_model, rf_param_grid, cv=3, scoring="neg_root_mean_squared_error", n_jobs=-1, verbose=1
)
rf_grid_search.fit(X_train_transformed_tree, y_train_original)

# Get best model & best parameters
best_rf_model = rf_grid_search.best_estimator_
best_rf_params = rf_grid_search.best_params_

# Predict using best model
y_pred_rf_best = best_rf_model.predict(X_val_transformed_tree)

# Compute RMSE for best RF model
rmse_rf_best = np.sqrt(mean_squared_error(y_val_original, y_pred_rf_best))

# Output best parameters and RMSE
best_rf_params, rmse_rf_best

In [ ]:
# Define the parameter grid for XGBoost
xgb_param_grid = {
    "n_estimators": [100, 150, 200],  # Number of boosting rounds
    "max_depth": [4, 6, 10],  # Depth of trees
    "learning_rate": [0.01, 0.05, 0.1],  # Step size for each tree
}

# Initialize XGBoost
xgb_model = XGBRegressor(objective="reg:squarederror", random_state=77)

# Perform Grid Search with Cross-Validation
xgb_grid_search = GridSearchCV(
    xgb_model, xgb_param_grid, cv=3, scoring="neg_root_mean_squared_error", verbose=1
)
xgb_grid_search.fit(X_train_transformed_tree, y_train_original)

# Get best model & best parameters
best_xgb_model = xgb_grid_search.best_estimator_
best_xgb_params = xgb_grid_search.best_params_

# Predict using best XGBoost model
y_pred_xgb_best = best_xgb_model.predict(X_val_transformed_tree)

# Compute RMSE for best XGBoost model
rmse_xgb_best = np.sqrt(mean_squared_error(y_val_original, y_pred_xgb_best))

# Output best parameters and RMSE
best_xgb_params, rmse_xgb_best

# Final prediction
We can see that the RF model is the best model. We'll choose it to generate our final predictions!

In [ ]:
# Initialize the best RF model with optimal hyperparameters from GridSearchCV
best_rf_model = RandomForestRegressor(
    max_depth=best_rf_params["max_depth"],
    n_estimators=best_rf_params["n_estimators"],
    random_state=42
)

# Train the final RF model on the full dataset
best_rf_model.fit(X_train_transformed_tree, y_train_original)

In [ ]:
# Apply the same preprocessing steps, use the trained transformer
X_test_transformed = preprocessor_tree.transform(df_test)  

In [ ]:
# Predict on the test set
test_predictions = best_rf_model.predict(X_test_transformed)

In [ ]:
# Save predictions to CSV file
submission = pd.DataFrame({"id": df_test.index, "predicted_revenue": test_predictions})
submission.to_csv("final_rf_predictions.csv", index=False)

# Feature Importance
* Here I answer the question - What are the three most important features that contributed to the prediction?

In [ ]:
# Get feature importance from the trained RF model
feature_importance = best_rf_model.feature_importances_

# Create a DataFrame to store feature names and importance scores
feature_importance_df = pd.DataFrame({
    "Feature": X_train_transformed_tree.columns,
    "Importance": feature_importance
})

# Sort features by importance in descending order
feature_importance_df = feature_importance_df.sort_values(by="Importance", ascending=False)

# Display the top 3 most important features
top_3_features = feature_importance_df.head(3)['Feature']
print("The three most important features that contributed to the prediction are:")
list(top_3_features)


# Part 2 - Recommendation task
For this task, I'll use T-Learner (Two-Model Approach) <br>
How It Works: <br>
Two separate machine learning models are trained:<br>
Model_A: Predicts revenue if Treatment A is given.<br>
Model_B: Predicts revenue if Treatment B is given.<br>
The treatment effect is estimated by comparing the predictions from the two models.

In [ ]:
# Loading the train data
train_df = pd.read_csv('train_home_assignment.csv', index_col=0)

# Loading the test data
test_df = pd.read_csv('test_home_assignment.csv', index_col=0)

In [ ]:
# Define features and target
X = train_df.drop(columns=["org_price_usd_following_30_days_after_impact", "org_price_usd_following_30_days"])
y = train_df["org_price_usd_following_30_days_after_impact"]

In [ ]:
# Split into training & validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Separate datasets based on treatment
X_train_A = X_train[X_train["treatment"] == 2].drop(columns=["treatment"])
y_train_A = y_train[X_train["treatment"] == 2]
X_val_A = X_val[X_val["treatment"] == 2].drop(columns=["treatment"])
y_val_A = y_val[X_val["treatment"] == 2]

X_train_B = X_train[X_train["treatment"] == 10].drop(columns=["treatment"])
y_train_B = y_train[X_train["treatment"] == 10]
X_val_B = X_val[X_val["treatment"] == 10].drop(columns=["treatment"])
y_val_B = y_val[X_val["treatment"] == 10]

In [ ]:
# Train separate models for Treatment A and B (Using the best parameters from the GridSearch above)
model_A = RandomForestRegressor(
    max_depth=best_rf_params["max_depth"],
    n_estimators=best_rf_params["n_estimators"],
    random_state=42
)
model_B = RandomForestRegressor(
    max_depth=best_rf_params["max_depth"],
    n_estimators=best_rf_params["n_estimators"],
    random_state=42
)

In [ ]:
# Train each model on its respective treatment group
model_A.fit(X_train_A, y_train_A)
model_B.fit(X_train_B, y_train_B)

In [ ]:
# Predict using best model
y_pred_A = model_A.predict(X_val_A)

# Compute RMSE for best RF model
rmse_rf_best = np.sqrt(mean_squared_error(y_val_A, y_pred_A))

# Output RMSE
rmse_rf_best

In [ ]:
# Predict using best model
y_pred_B = model_B.predict(X_val_B)

# Compute RMSE for best RF model
rmse_rf_best = np.sqrt(mean_squared_error(y_val_B, y_pred_B))

# Output RMSE
rmse_rf_best

In [ ]:
# Prepare test data (remove treatment column for prediction)
X_test = test_df.drop(columns=["treatment"], errors="ignore")

In [ ]:
# Create two copies of test data (one for each treatment)
X_test_A = X_test.copy()
X_test_B = X_test.copy()

# Predict revenue under each treatment
pred_A = model_A.predict(X_test_A)
pred_B = model_B.predict(X_test_B)

In [ ]:
# Assign the best treatment based on highest predicted revenue - in a new column
test_df["optimal_treatment"] = np.where(pred_A > pred_B, 2, 10)

In [ ]:
# Save results
test_df.to_csv("recommended_treatments_t_learner.csv", index=False)

In [ ]:
# Extract feature importance from the trained treatment prediction models
feature_importance_A = model_A.feature_importances_
feature_importance_B = model_B.feature_importances_

# Create DataFrames to store feature importance scores for each treatment model
feature_importance_A_df = pd.DataFrame({
    "Feature": X_train_A.columns,
    "Importance_A": feature_importance_A
}).sort_values(by="Importance_A", ascending=False)

feature_importance_B_df = pd.DataFrame({
    "Feature": X_train_B.columns,
    "Importance_B": feature_importance_B
}).sort_values(by="Importance_B", ascending=False)

# Merge both feature importance rankings
feature_importance_combined = feature_importance_A_df.merge(
    feature_importance_B_df, on="Feature", how="inner"
)

# Compute average importance score across both models
feature_importance_combined["Avg_Importance"] = (
    feature_importance_combined["Importance_A"] + feature_importance_combined["Importance_B"]
) / 2

# Sort by average importance to get the most influential features
top_3_treatment_features = feature_importance_combined.sort_values(
    by="Avg_Importance", ascending=False
).head(3)

# Output the top 3 most important features influencing treatment decisions
top_3_treatment_features

# THE END :)